# ライブラリのインポート

In [1]:
import sqlite3
import pandas as pd

# まとめ

In [2]:
# DBの作成・接続
def get_and_create_connect_db(dbname='KEIBA.db'):
    conn = sqlite3.connect(dbname)
    return conn

# csvからテーブルの作成・追加
def create_or_insert_table_by_csv(dbname, csv_file, table_name='sample'):
    df = pd.read_csv(csv_file, index_col=0)
    conn = get_and_create_connect_db(dbname)
    cur = conn.cursor()
    df.to_sql(table_name, conn, if_exists='append')
    cur.close()
    conn.close()
    
# DataFrameからテーブルの作成・追加
def create_or_insert_table_by_df(dbname, df, table_name='sample'):
    conn = get_and_create_connect_db(dbname)
    cur = conn.cursor()
    df.to_sql(table_name, conn, if_exists='append')
    cur.close()
    conn.close()

# DBからクエリでデータ抽出
def get_df_by_query(dbname, query='SELECT * FROM sample'):
    conn = get_and_create_connect_db(dbname)
    cur = conn.cursor()
    df = pd.read_sql(query, conn)
    cur.close()
    conn.close()
    return df

# メソッドの動作確認

In [3]:
conn = get_and_create_connect_db(dbname='hogehoge.db')

In [4]:
csv_file = 'sample.csv'

In [5]:
create_or_insert_table_by_csv(dbname, csv_file, table_name='hoge')

NameError: name 'dbname' is not defined

In [ ]:
df = pd.read_csv("sample2.csv", index_col=0)
create_or_insert_table_by_df(dbname, df, 'hoge')

In [6]:
get_df_by_query(dbname, 'select * from hoge')

NameError: name 'dbname' is not defined

# メソッド化する前の動作確認

## DBの作成（存在していたら接続）

In [7]:
# KEIBA.dbを作成する
dbname = 'KEIBA.db'
conn = sqlite3.connect(dbname)

# データベースへのコネクションを閉じる。(必須)
conn.close()

## テーブルの作成

In [8]:
conn = sqlite3.connect(dbname)
# sqliteを操作するカーソルオブジェクトを作成
cur = conn.cursor()

# テーブルの作成（racesというテーブルを作成する例）
cur.execute(
    'CREATE TABLE races(id INTEGER PRIMARY KEY AUTOINCREMENT, name STRING)'
)

# データベースへコミット。これで変更が反映される。
conn.commit()
conn.close()

## csvからテーブルの作成

In [9]:
url = 'https://db.netkeiba.com/race/202205050301/'
dfs = pd.read_html(url)
dfs[0].to_csv('sample.csv')

In [10]:
# csvには、1列目にyear, 2列目にmonth, 3列目にdayが入っているとする。
df = pd.read_csv("sample.csv", index_col=0)

# カラム名（列ラベル）を作成。csv file内にcolumn名がある場合は、下記は不要
# pandasが自動で1行目をカラム名として認識してくれる。
# df.columns = ['year', 'month', 'day']

conn = sqlite3.connect(dbname)
cur = conn.cursor()

# if_existsで、もしすでにexpenseが存在していたら、書き換えるように指示
df.to_sql('sample', conn, if_exists='append')

cur.close()
conn.close()

## DBからpandasで読み取る

In [11]:
conn = sqlite3.connect(dbname)
cur = conn.cursor()

# dbをpandasで読み出す。
df = pd.read_sql('SELECT * FROM sample', conn)

cur.close()
conn.close()

In [12]:
df.head()

,index,着 順,枠 番,馬 番,馬名,性齢,斤量,騎手,タイム,着差,単勝,人 気,馬体重,調教師
0,0,1,2,3,ミラクルキャッツ,牝2,54,レーン,1:39.8,None,3.6,2,456(+2),[東] 堀宣行
1,1,2,6,11,ニシノパプルブリリ,牝2,54,津村明秀,1:40.0,1.1/4,85.1,10,460(0),[東] 矢野英一
2,2,3,8,16,リリックス,牝2,54,松岡正海,1:40.8,5,10.8,5,484(-4),[東] 蛯名正義
3,3,4,4,8,ラピッドベル,牝2,54,戸崎圭太,1:41.0,1.1/4,2.8,1,496(-8),[西] 牧浦充徳
4,4,5,8,15,エテルネル,牝2,54,石川裕紀,1:41.7,4,5.1,3,462(-4),[東] 池上昌和


## DBからSQLでデータを抽出

In [13]:
conn = sqlite3.connect(dbname)
cur = conn.cursor()
select_sql = 'SELECT * FROM sample'
cur.execute(select_sql)
data = cur.fetchall()

In [14]:
# DataFrame
columns = [s[0] for s in cur.description]
df = pd.DataFrame(data, columns=columns)
cur.close()
conn.close()

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   index   16 non-null     int64  
 1   着 順     16 non-null     int64  
 2   枠 番     16 non-null     int64  
 3   馬 番     16 non-null     int64  
 4   馬名      16 non-null     object 
 5   性齢      16 non-null     object 
 6   斤量      16 non-null     int64  
 7   騎手      16 non-null     object 
 8   タイム     16 non-null     object 
 9   着差      15 non-null     object 
 10  単勝      16 non-null     float64
 11  人 気     16 non-null     int64  
 12  馬体重     16 non-null     object 
 13  調教師     16 non-null     object 
dtypes: float64(1), int64(6), object(7)
memory usage: 1.9+ KB


## データの追加

In [16]:
url = 'https://db.netkeiba.com/race/202205050302/'
dfs = pd.read_html(url)
dfs[0].to_csv('sample2.csv')

In [17]:
# csvには、1列目にyear, 2列目にmonth, 3列目にdayが入っているとする。
df = pd.read_csv("sample2.csv", index_col=0)

# カラム名（列ラベル）を作成。csv file内にcolumn名がある場合は、下記は不要
# pandasが自動で1行目をカラム名として認識してくれる。
# df.columns = ['year', 'month', 'day']

conn = sqlite3.connect(dbname)
cur = conn.cursor()

# if_existsで、もしすでに存在していたら追加するように指示
df.to_sql('sample', conn, if_exists='append')

cur.close()
conn.close()

In [18]:
conn = sqlite3.connect(dbname)
cur = conn.cursor()
select_sql = 'SELECT * FROM sample'
cur.execute(select_sql)
data = cur.fetchall()
df = pd.DataFrame(data, columns=columns)

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28 entries, 0 to 27
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   index   28 non-null     int64  
 1   着 順     28 non-null     int64  
 2   枠 番     28 non-null     int64  
 3   馬 番     28 non-null     int64  
 4   馬名      28 non-null     object 
 5   性齢      28 non-null     object 
 6   斤量      28 non-null     int64  
 7   騎手      28 non-null     object 
 8   タイム     28 non-null     object 
 9   着差      26 non-null     object 
 10  単勝      28 non-null     float64
 11  人 気     28 non-null     int64  
 12  馬体重     28 non-null     object 
 13  調教師     28 non-null     object 
dtypes: float64(1), int64(6), object(7)
memory usage: 3.2+ KB


# データの確認

In [20]:
df

,index,着 順,枠 番,馬 番,馬名,性齢,斤量,騎手,タイム,着差,単勝,人 気,馬体重,調教師
0,0,1,2,3,ミラクルキャッツ,牝2,54,レーン,1:39.8,None,3.6,2,456(+2),[東] 堀宣行
1,1,2,6,11,ニシノパプルブリリ,牝2,54,津村明秀,1:40.0,1.1/4,85.1,10,460(0),[東] 矢野英一
2,2,3,8,16,リリックス,牝2,54,松岡正海,1:40.8,5,10.8,5,484(-4),[東] 蛯名正義
3,3,4,4,8,ラピッドベル,牝2,54,戸崎圭太,1:41.0,1.1/4,2.8,1,496(-8),[西] 牧浦充徳
4,4,5,8,15,エテルネル,牝2,54,石川裕紀,1:41.7,4,5.1,3,462(-4),[東] 池上昌和
5,5,6,3,6,ヒラリ,牝2,54,ルメール,1:41.8,クビ,7.9,4,460(+4),[西] 武幸四郎
6,6,7,5,9,ミズカガミ,牝2,53,永野猛蔵,1:41.9,1/2,143.9,13,442(-2),[東] 土田稔
7,7,8,5,10,リーア,牝2,54,横山和生,1:41.9,アタマ,60.1,9,444(+2),[東] 小西一男
8,8,9,1,2,ポポラーレウーノ,牝2,51,西塚洸二,1:41.9,クビ,21.6,6,472(+12),[東] 竹内正洋
9,9,10,3,5,ラディーヴァ,牝2,54,木幡巧也,1:42.0,1/2,306.9,15,458(-2),[東] 武市康男
